In [493]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import optuna
from sklearn import metrics
import sklearn.cluster
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

file = '..\data\external\Legally_Operating_Businesses.csv'
df = pd.read_csv(file)
df = df[df['Latitude'].isna() == False]
df = df[df['Business Name'].isna()==False]
df = df[df['Address State']=='NY']
df['License Creation Date'] = pd.to_datetime(df['License Creation Date'], format = '%m/%d/%Y')
df['License Expiration Date'] = pd.to_datetime(df['License Expiration Date'], format = '%m/%d/%Y')
df['License Status'] = df['License Status']=='Active'
df['License Status'] = df['License Status'].astype('int')
drop_cols = ['DCA License Number', 'License Type', 'Business Name', 'Business Name 2', 'Address Building', 'Address Street Name', 'Secondary Address Street Name', 'Address City', 'Address State', 'Address ZIP', 'Contact Phone Number', 'Address Borough','Borough Code','Community Board','Council District','BIN','BBL','NTA','Census Tract','Detail','Location']
df.drop(labels = drop_cols, axis=1, inplace = True)
df = df[df['Longitude'] > -76]
df.reset_index(drop = True, inplace = True)
df['date_diffs']=(df['License Expiration Date']-df['License Creation Date']).dt.days
df=df[df['date_diffs']>0]
df['Start_date']=(df['License Creation Date']-np.min(df['License Creation Date'])).dt.days
df_small = df.sample(frac=0.10)
df.head()

C:\Users\JackW\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,License Expiration Date,License Status,License Creation Date,Industry,Longitude,Latitude,date_diffs,Start_date
0,2022-06-30,1,2007-01-02,Electronic & Appliance Service,-73.835446,40.838469,5658.0,10935
1,2022-05-01,1,2018-10-31,Employment Agency,-73.795002,40.710524,1278.0,15255
2,2022-06-30,1,2015-10-27,Electronic & Appliance Service,-74.010425,40.645018,2438.0,14155
3,2022-05-01,1,2012-07-13,Employment Agency,-73.969382,40.792751,3579.0,12954
4,2022-06-30,1,2012-01-10,Electronic & Appliance Service,-73.825319,40.733833,3824.0,12769


In [494]:
class Cluster_Adder():
    def __init__(self, K):
        self.K = K
    
    def fit(self, X , y):
        
        kmeans = sklearn.cluster.KMeans(n_clusters=self.K)
        self.assigned_cluster=kmeans.fit_predict(X[:,0:3])
        means=np.zeros(self.K)
        for i in range(self.K):
            means[i]=y[self.assigned_cluster==i].mean()
        self.means=means
        self.kmeans=kmeans
        return self
    
    def transform(self, X):
        
        # Use the already predicted clusters to save time if this is what we trained our clusters on
        if (X.shape[0]==len(self.assigned_cluster)):
            cluster_col=np.zeros(X.shape[0])
            for i in range(self.K):
                cluster_col[self.assigned_cluster==i]=self.means[i]
            return np.column_stack((X,cluster_col))
    
        # Otherwise we predict the clusters of the test points
        assigned_cluster=self.kmeans.predict(X[:,0:3])

        cluster_col=np.zeros(X.shape[0])
        
        for i in range(self.K):
            cluster_col[assigned_cluster==i]=self.means[i]
        return np.column_stack((X,cluster_col))

In [495]:
# This also adds a column with the mean for the cluster, but deletes the location and time data
class Cluster_Adder_With_Deletion():
    def __init__(self, K):
        self.K = K
    
    def fit(self, X , y):
        
        kmeans = sklearn.cluster.KMeans(n_clusters=self.K)
        self.assigned_cluster=kmeans.fit_predict(X[:,0:3])
        means=np.zeros(self.K)
        for i in range(self.K):
            means[i]=y[self.assigned_cluster==i].mean()
        self.means=means
        self.kmeans=kmeans
        return self
    
    def transform(self, X):
        
        # Use the already predicted clusters to save time if this is what we trained our clusters on
        if (X.shape[0]==len(self.assigned_cluster)):
            cluster_col=np.zeros(X.shape[0])
            for i in range(self.K):
                cluster_col[self.assigned_cluster==i]=self.means[i]
            # The subset here deletes the time and location data
            return np.column_stack((X[:,3:],cluster_col))
    
        # Otherwise we predict the clusters of the test points
        assigned_cluster=self.kmeans.predict(X[:,0:3])

        cluster_col=np.zeros(X.shape[0])
        
        for i in range(self.K):
            cluster_col[assigned_cluster==i]=self.means[i]
        # The subset here deletes the time and location data
        return np.column_stack((X[:,3:],cluster_col))

In [537]:
def objective_clusters(trial):
    
    data = df_small[['Start_date','Longitude','Latitude','License Status','Industry']]
    target = df_small[['date_diffs']]
    
    K = trial.suggest_int("K", 2, 200)
    
    alpha=trial.suggest_uniform('alpha',0.1,1)
    
    beta=trial.suggest_uniform('beta',0.1,1)
    
    parameter = {
      'max_depth':trial.suggest_int('depth', 3, 5), # show integer parameters between 3 and 5 for depth
      'min_child_weight':trial.suggest_int('childweight',0,5), # show integer parameters between 0 and 5 for childweight
      'learning_rate':trial.suggest_loguniform('ourlearning_rate',0.05,0.6), # set a log distribution between 0.05 and 0.5 for learning rate
      'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9), # set a uniformly distributed numbers between 0.4 and 0.9 for colsample_bytree
      'subsample':trial.suggest_uniform('sample',0.4,0.9)
    }
    
    # preprocessor to avoid data leakage
    preprocessor = ColumnTransformer(transformers = [('scaler1', MinMaxScaler((0,alpha)),['Longitude', 'Latitude']),('scaler2', MinMaxScaler((0,beta)),['Start_date']), ('onehot', OneHotEncoder(sparse=False,handle_unknown = 'ignore'), ['Industry'])])
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-erorr")
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),('clusterer',Cluster_Adder_With_Deletion(K)),('model', xgb.XGBRegressor(**parameter))])

    return np.mean(cross_val_score(pipeline, data, target, cv=3,))


In [538]:
study_clusters = optuna.create_study(direction='maximize',study_name='Clusters')
study_clusters.optimize(objective_clusters,n_trials=25)

[I 2022-06-03 22:01:36,335] A new study created in memory with name: Clusters
[I 2022-06-03 22:02:17,596] Trial 0 finished with value: 0.24369537330535843 and parameters: {'K': 162, 'alpha': 0.8699654924298679, 'beta': 0.8416441544579751, 'depth': 3, 'childweight': 0, 'ourlearning_rate': 0.09778654832531708, 'colsample_bytree': 0.4147372085475847, 'sample': 0.6438415020658207}. Best is trial 0 with value: 0.24369537330535843.
[I 2022-06-03 22:02:28,110] Trial 1 finished with value: 0.14263120922889536 and parameters: {'K': 19, 'alpha': 0.8891442648514497, 'beta': 0.35579996124309254, 'depth': 4, 'childweight': 3, 'ourlearning_rate': 0.05925592734816416, 'colsample_bytree': 0.6842692799236261, 'sample': 0.8871381136360678}. Best is trial 0 with value: 0.24369537330535843.
[I 2022-06-03 22:02:52,407] Trial 2 finished with value: 0.2616181499880961 and parameters: {'K': 66, 'alpha': 0.25961498289876717, 'beta': 0.8431230221074335, 'depth': 3, 'childweight': 0, 'ourlearning_rate': 0.124800

[I 2022-06-03 22:11:11,387] Trial 23 finished with value: 0.2629510527814231 and parameters: {'K': 54, 'alpha': 0.1070298561054032, 'beta': 0.8793746108325665, 'depth': 5, 'childweight': 4, 'ourlearning_rate': 0.08388599772744885, 'colsample_bytree': 0.656040210819067, 'sample': 0.6818903627208958}. Best is trial 13 with value: 0.26591565278518065.
[I 2022-06-03 22:11:24,922] Trial 24 finished with value: 0.23864875765692095 and parameters: {'K': 22, 'alpha': 0.38361989352589404, 'beta': 0.7609551240876715, 'depth': 5, 'childweight': 3, 'ourlearning_rate': 0.1439010800083545, 'colsample_bytree': 0.8100262473827385, 'sample': 0.7578625332452301}. Best is trial 13 with value: 0.26591565278518065.


In [531]:
def objective_no_clusters(trial):
    
    data = df_small[['Start_date','Longitude','Latitude','License Status','Industry']]
    target = df_small[['date_diffs']]
    
    alpha=trial.suggest_uniform('alpha',0.1,1)
    
    beta=trial.suggest_uniform('beta',0.1,1)
    
    parameter = {
      'max_depth':trial.suggest_int('depth', 3, 5), # show integer parameters between 3 and 5 for depth
      'min_child_weight':trial.suggest_int('childweight',0,5), # show integer parameters between 0 and 5 for childweight
      'learning_rate':trial.suggest_loguniform('ourlearning_rate',0.05,0.6), # set a log distribution between 0.05 and 0.5 for learning rate
      'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9), # set a uniformly distributed numbers between 0.4 and 0.9 for colsample_bytree
      'subsample':trial.suggest_uniform('sample',0.4,0.9)
    }
    
    # preprocessor to avoid data leakage
    preprocessor = ColumnTransformer(transformers = [('scaler1', MinMaxScaler((0,alpha)),['Longitude', 'Latitude']),('scaler2', MinMaxScaler((0,beta)),['Start_date']), ('onehot', OneHotEncoder(sparse=False,handle_unknown = 'ignore'), ['Industry'])])
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-erorr")
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', xgb.XGBRegressor(**parameter))])

    return np.mean(cross_val_score(pipeline, data, target, cv=3))


In [532]:
study_no_clusters = optuna.create_study(direction='maximize',study_name='No Clusters')
study_no_clusters.optimize(objective_no_clusters,n_trials=25)

[I 2022-06-03 21:53:36,365] A new study created in memory with name: No Clusters
[I 2022-06-03 21:53:38,503] Trial 0 finished with value: 0.22924773865689194 and parameters: {'alpha': 0.854084906587587, 'beta': 0.4855154232172746, 'depth': 4, 'childweight': 0, 'ourlearning_rate': 0.3247063181156315, 'colsample_bytree': 0.7680860773013716, 'sample': 0.7350400907839559}. Best is trial 0 with value: 0.22924773865689194.
[I 2022-06-03 21:53:40,504] Trial 1 finished with value: 0.2573266376473112 and parameters: {'alpha': 0.6272353504758643, 'beta': 0.8702833613969365, 'depth': 4, 'childweight': 4, 'ourlearning_rate': 0.12296453138158003, 'colsample_bytree': 0.6784496264661272, 'sample': 0.42129718625222634}. Best is trial 1 with value: 0.2573266376473112.
[I 2022-06-03 21:53:42,497] Trial 2 finished with value: 0.2590366579059769 and parameters: {'alpha': 0.40668507100781437, 'beta': 0.1823982045539866, 'depth': 4, 'childweight': 5, 'ourlearning_rate': 0.05025128275895816, 'colsample_bytre

[I 2022-06-03 21:55:25,883] Trial 23 finished with value: 0.263275193542402 and parameters: {'alpha': 0.2144506329797492, 'beta': 0.5705564338017767, 'depth': 5, 'childweight': 2, 'ourlearning_rate': 0.06742075184804985, 'colsample_bytree': 0.715227838592398, 'sample': 0.573719451907023}. Best is trial 12 with value: 0.2649414620758183.
[I 2022-06-03 21:55:30,794] Trial 24 finished with value: 0.2649666455657007 and parameters: {'alpha': 0.2050545815704506, 'beta': 0.5820437900729117, 'depth': 5, 'childweight': 1, 'ourlearning_rate': 0.07877940303841176, 'colsample_bytree': 0.6259669172923449, 'sample': 0.5933566047260158}. Best is trial 24 with value: 0.2649666455657007.


In [529]:
def objective_empty(trial):
    
    data = df_small[['License Status','Industry']]
    target = df_small[['date_diffs']]
    parameter = {
      'max_depth':trial.suggest_int('depth', 3, 5), # show integer parameters between 3 and 5 for depth
      'min_child_weight':trial.suggest_int('childweight',0,5), # show integer parameters between 0 and 5 for childweight
      'learning_rate':trial.suggest_loguniform('ourlearning_rate',0.05,0.6), # set a log distribution between 0.05 and 0.5 for learning rate
      'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9), # set a uniformly distributed numbers between 0.4 and 0.9 for colsample_bytree
      'subsample':trial.suggest_uniform('sample',0.4,0.9)
    }
    
    # preprocessor to avoid data leakage
    preprocessor = ColumnTransformer(transformers = [('onehot', OneHotEncoder(sparse=False,handle_unknown = 'ignore'), ['Industry'])])
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-erorr")
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', xgb.XGBRegressor(**parameter))])

    return np.mean(cross_val_score(pipeline, data, target, cv=3))


In [530]:
study_empty = optuna.create_study(direction='maximize',study_name='Empty')
study_empty.optimize(objective_empty,n_trials=25)

[I 2022-06-03 21:52:42,944] A new study created in memory with name: Empty
[I 2022-06-03 21:52:44,474] Trial 0 finished with value: 0.048790136675298044 and parameters: {'depth': 4, 'childweight': 5, 'ourlearning_rate': 0.18167838611750486, 'colsample_bytree': 0.6418584114077657, 'sample': 0.502936623145783}. Best is trial 0 with value: 0.048790136675298044.
[I 2022-06-03 21:52:45,870] Trial 1 finished with value: 0.049814245318445104 and parameters: {'depth': 3, 'childweight': 2, 'ourlearning_rate': 0.130863215118722, 'colsample_bytree': 0.7622182137407698, 'sample': 0.7645340904563582}. Best is trial 1 with value: 0.049814245318445104.
[I 2022-06-03 21:52:47,272] Trial 2 finished with value: 0.048272769581729956 and parameters: {'depth': 3, 'childweight': 1, 'ourlearning_rate': 0.0811369853638437, 'colsample_bytree': 0.8153794564593015, 'sample': 0.8455972872043664}. Best is trial 1 with value: 0.049814245318445104.
[I 2022-06-03 21:52:48,527] Trial 3 finished with value: 0.049540397